In [29]:
import pandas as pd
import numpy as np

file = pd.read_csv('fifa21_train.csv')

In [30]:
def clean(file):
    file=file.rename(columns={
    'FK Accuracy':'Free kick Accuracy',
    'GK Diving':'Goalkeeper Diving',
    'GK Handling':'Goalkeeper Handling',
    'GK Kicking':'Goalkeeper Kicking',
    'GK Positioning':'Goalkeeper Position',
    'GK Reflexes':'Goalkeeper Reflexes',
    'W/F':'Weak foot',
    'SM':'Skill Moves',
    'A/W':'Attacking Workrate',
    'D/W':'Defensive Workrate',
    'IR':'International Reputation',
    'PAC':'Pace',
    'SHO':'Shooting',
    'PAS':'Passing',
    'DRI':'Dribbling',
    'DEF':'Defending',
    'PHY':'Physicality',
    'LS':'LS: Left Striker',
    'ST':'ST: Striker',
    'RS':'RS: Right Striker',
    'LW':'LW: Left Winger',
    'LF':'LF: Left Forward',
    'CF':'CF: Center Forward',
    'RF':'RF: Right Forward',
    'RW':'RW: Right Winger',
    'LAM':'LAM: Left Attacking Midfielder',
    'CAM':'CAM: Center Attacking Midfielder',
    'RAM':'RAM: Right Attacking Midfielder',
    'LM':'LM: Left Midfielder',
    'LCM':'LCM: Left Center Midfielder',
    'CM':'CM: Center Midfielder',
    'RCM':'RCM: Right Center Midfielder',
    'RM':'RM: Right Midfielder',
    'LWB':'LWB: Left Wing Back',
    'LDM':'LDM: Left Defensive Midfielder',
    'CDM':'CDM: Center Defensive Midfielder',
    'RDM':'RDM: Right Defensive Midfielder',
    'RWB':'RWB: Right Wing Back',
    'LB':'LB: Left Back',
    'LCB':'LCB: Left Center Back',
    'CB':'CB: Center Back',
    'RCB':'RCB: Right Center Back',
    'RB':'RB: Right Back',
    'GK':'GK: Goalkeeper',
    'OVA':'OVA: Overall Average'
    })
    file = file[file['Club'].isna() == False]
    file['Position'] = file['Position'].fillna(file['BP']) 
    file['total position'] = file['Position'].apply(lambda x: len(x.split(' '))) 
    file.drop('Position', axis= 1, inplace = True)
    file.drop('ID', axis= 1, inplace = True)
    file.drop('Loan Date End',axis=1, inplace= True)
    file = file[file['Joined'].isna() == False] 
    file['Composure'] = file['Composure'].fillna(file['Composure'].mean()) 
    file['Attacking Workrate'] = file['Attacking Workrate'].str.replace("High", '3')
    file['Attacking Workrate'] = file['Attacking Workrate'].str.replace("Medium", '2')
    file['Attacking Workrate'] = file['Attacking Workrate'].str.replace("Low", '1')
    file['Defensive Workrate'] = file['Defensive Workrate'].str.replace("High", '3')
    file['Defensive Workrate'] = file['Defensive Workrate'].str.replace("Medium", '2')
    file['Defensive Workrate'] = file['Defensive Workrate'].str.replace("Low", '1')
    file['Attacking Workrate'] = file['Attacking Workrate'].fillna(file['Attacking Workrate'].median()).astype('int')
    file['Defensive Workrate'] = file['Defensive Workrate'].fillna(file['Defensive Workrate'].median()).astype('int')
    file['Wage']=file['Wage'].str.replace("€",'')
    file['Wage']=file['Wage'].str.replace("K",'000')
    file['Wage']=file['Wage'].str.replace("M",'000000')
    file['Wage']=file['Wage'].str.replace(".",'')
    file['Value']=file['Value'].str.replace("€",'')
    file['Value']=file['Value'].str.replace("K",'000')
    file['Value']=file['Value'].str.replace("M",'000000')
    file['Value']=file['Value'].str.replace(".",'')
    file['Release Clause']=file['Release Clause'].str.replace("€",'')
    file['Release Clause']=file['Release Clause'].str.replace("K",'000')
    file['Release Clause']=file['Release Clause'].str.replace("M",'000000')
    file['Release Clause']=file['Release Clause'].str.replace(".",'')
    coltoremove = file.columns[-30:-2]
    for columns in coltoremove:
        file.drop(columns, axis= 1, inplace= True)
    def change_height(ht):
        ftin = ht.split("'")
        feet = int(ftin[0])
        inches = float(ftin[1].split('"')[0])
        return (12*feet) + inches
    file['Height'] = file['Height'].apply(lambda x: change_height(x))
    file['Weight'] = file['Weight'].apply(lambda x: x.replace('lbs', '')).astype('int')
    file['Joined'] = pd.to_datetime(file['Joined']).dt.year
    file['Wage'] = pd.to_numeric(file['Wage'])
    file['Value'] = pd.to_numeric(file['Value'])
    file['Release Clause'] = pd.to_numeric(file['Release Clause'])
    file['Weak foot'] = file['Weak foot'].str.rstrip('★').astype('int')
    file['Skill Moves'] = file['Skill Moves'].str.rstrip('★').astype('int')
    file['International Reputation'] = file['International Reputation'].str.rstrip('★').astype('int')
    file.drop('Contract', axis= 1, inplace= True)
    file.drop('Name', axis= 1, inplace= True)
    file.drop('Team & Contract', axis = 1, inplace= True)
    file.drop('Club', axis = 1, inplace= True)
    file.drop('Nationality', axis = 1, inplace= True)
    categoricals = file.select_dtypes('object')
    numericals = file._get_numeric_data().reset_index()
    from sklearn.preprocessing import OneHotEncoder

    encoder = OneHotEncoder(drop='first').fit(categoricals) 
    encoded = encoder.transform(categoricals).toarray() 
    encoded_columns = OneHotEncoder(drop='first').fit(categoricals).get_feature_names(input_features=categoricals.columns) # this allows us to get the columns names for our encoded array

    onehot_encoded_X = pd.DataFrame(encoded, columns = encoded_columns) 
    onehot_encoded_X.reset_index()
    new_X = pd.concat([onehot_encoded_X, numericals], axis=1)
    return new_X

In [31]:
file.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Columns: 101 entries, ID to OVA
dtypes: float64(10), int64(43), object(48)
memory usage: 39.7 MB


In [32]:
file = clean(file)

C:\Users\A.M\AppData\Local\Temp\ipykernel_10520\2077552913.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  file['Wage']=file['Wage'].str.replace(".",'')
C:\Users\A.M\AppData\Local\Temp\ipykernel_10520\2077552913.py:72: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  file['Value']=file['Value'].str.replace(".",'')
C:\Users\A.M\AppData\Local\Temp\ipykernel_10520\2077552913.py:76: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  file['Release Clause']=file['Release Clause'].str.replace(".",'')
C:\Users\A.M\anaconda3\lib\sit

In [33]:
pd.set_option('display.max_columns', None) #max columns
file.head(5)

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,BP_RM,BP_RW,BP_RWB,BP_ST,foot_Right,index,Age,Height,Weight,Growth,Joined,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,Free kick Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,Goalkeeper Diving,Goalkeeper Handling,Goalkeeper Kicking,Goalkeeper Position,Goalkeeper Reflexes,Total Stats,Base Stats,Weak foot,Skill Moves,Attacking Workrate,Defensive Workrate,International Reputation,Pace,Shooting,Passing,Dribbling,Defending,Physicality,OVA: Overall Average,total position
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,26,69.0,161,1,2015,525000,4000,801000,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,3,2,1,69,51,63,63,51,60,64,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,30,72.0,159,0,2015,85000000,23000,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,3,1,2,83,75,68,82,33,71,77,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,33,64.0,134,0,2019,9000000,49000,153000000,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,3,2,2,80,77,78,86,27,56,80,2
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,22,70.0,152,13,2016,275000,4000,694000,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,2,2,1,57,44,54,57,57,60,59,2
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,23,71.0,150,8,2018,725000,2000,14000000,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,1,2,1,66,44,60,64,60,66,65,2
